In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# AutoML training image image classification model for export to edge



## Overview


This tutorial demonstrates how to use the Vertex AI SDK to create image classification models to export as an Edge model using AutoML.

### Objective

In this tutorial, you create an AutoML classification model from a Python script using the Vertex SDK, and then export the model as an Edge model in TFLite format. You can alternatively create models with AutoML using the `gcloud` command-line tool or online using the Cloud Console.

This tutorial uses the following Google Cloud ML services:

- Vertex AI `Datasets`
- AutoML Image

The steps performed include:

- Create a Vertex `Dataset` resource.
- Train the model.
- Export the `Edge` model from the `Model` resource to Cloud Storage.
- Download the model locally.
- Make a local prediction.

### Dataset

The dataset used for this tutorial is a visual inspection auto dataset containing defective and non-defective items

### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage

Learn about [Vertex AI
pricing](https://cloud.google.com/vertex-ai/pricing) and [Cloud Storage
pricing](https://cloud.google.com/storage/pricing), and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

## Installation

Install the latest version of Vertex AI SDK for Python.

In [1]:
import os


# Google Cloud Notebook
if os.path.exists("/opt/deeplearning/metadata/env_version"):
    USER_FLAG = "--user"
else:
    USER_FLAG = ""

! pip3 install --upgrade google-cloud-aiplatform $USER_FLAG


  Using cached google_cloud_aiplatform-1.24.0-py2.py3-none-any.whl (2.5 MB)
  Using cached protobuf-4.22.3-cp37-abi3-manylinux2014_x86_64.whl (302 kB)
  Using cached packaging-21.3-py3-none-any.whl (40 kB)
  Using cached google_cloud_resource_manager-1.9.1-py2.py3-none-any.whl (276 kB)
  Using cached google_cloud_resource_manager-1.8.1-py2.py3-none-any.whl (235 kB)
  Using cached google_cloud_resource_manager-1.8.0-py2.py3-none-any.whl (235 kB)
  Using cached google_cloud_resource_manager-1.7.0-py2.py3-none-any.whl (235 kB)
  Using cached google_cloud_resource_manager-1.6.3-py2.py3-none-any.whl (233 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.1
    Uninstalling protobuf-3.20.1:
ERROR: Could not install packages due to an OSError: [Errno 13] Permission denied: 'service.py'
Consider using the `--user` option or check the permissions.



### Colab only: Uncomment the following cell to restart the kernel

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

## Before you begin

### Set your project ID

**If you don't know your project ID**, try the following:
* Run `gcloud config list`.
* Run `gcloud projects list`.
* See the support page: [Locate the project ID](https://support.google.com/googleapi/answer/7014113)

In [2]:
PROJECT_ID = "pv-mlops-demo"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

Updated property [core/project].


#### Region

You can also change the `REGION` variable used by Vertex AI. Learn more about [Vertex AI regions](https://cloud.google.com/vertex-ai/docs/general/locations).

In [3]:
REGION = "us-central1"  # @param {type: "string"}

### Authenticate your Google Cloud account

Depending on your Jupyter environment, you may have to manually authenticate. Follow the relevant instructions below.

**1. Vertex AI Workbench**
* Do nothing as you are already authenticated.

**2. Local JupyterLab instance, uncomment and run:**

In [4]:
# ! gcloud auth login

**3. Colab, uncomment and run:**

In [5]:
# from google.colab import auth
# auth.authenticate_user()

**4. Service account or other**
* See how to grant Cloud Storage permissions to your service account at https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples.

### Create a Cloud Storage bucket

Create a storage bucket to store intermediate artifacts such as datasets.

In [6]:
BUCKET_URI = f"gs://automl-vision-workshop-{PROJECT_ID}"  # @param {type:"string"}

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [7]:
! gsutil mb -l $REGION $BUCKET_URI

Creating gs://automl-vision-workshop-pv-mlops-demo/...


### Set up variables

Next, set up some variables used throughout the tutorial.
### Import libraries and define constants

In [8]:
import google.cloud.aiplatform as aiplatform

## Initialize Vertex AI SDK for Python

Initialize the Vertex AI SDK for Python for your project and corresponding bucket.

In [9]:
aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

# Tutorial

Now you are ready to start creating your own AutoML image object detection model.

#### Location of Cloud Storage training data.

Now set the variable `IMPORT_FILE` to the location of the CSV index file in Cloud Storage.

In [10]:
IMPORT_FILE = "gs://pv-visual-inspection-ai-auto/visual_inspection/parts_label.csv"

#### Quick peek at your data

This tutorial uses a version of the Salads dataset that is stored in a public Cloud Storage bucket, using a CSV index file.

Start by doing a quick peek at the data. You count the number of examples by counting the number of rows in the CSV index file  (`wc -l`) and then peek at the first few rows.

In [11]:
if "IMPORT_FILES" in globals():
    FILE = IMPORT_FILES[0]
else:
    FILE = IMPORT_FILE

count = ! gsutil cat $FILE | wc -l
print("Number of Examples", int(count[0]))

print("First 10 rows")
! gsutil cat $FILE | head

Number of Examples 29
First 10 rows
gs://pv-visual-inspection-ai-auto/visual_inspection/defective/defect_1.jpeg,defective
gs://pv-visual-inspection-ai-auto/visual_inspection/defective/defect_2.jpeg,defective
gs://pv-visual-inspection-ai-auto/visual_inspection/defective/defect_3.jpeg,defective
gs://pv-visual-inspection-ai-auto/visual_inspection/defective/defect_4.jpeg,defective
gs://pv-visual-inspection-ai-auto/visual_inspection/defective/defect_5.jpeg,defective
gs://pv-visual-inspection-ai-auto/visual_inspection/defective/defect_6.jpeg,defective
gs://pv-visual-inspection-ai-auto/visual_inspection/defective/defect_7.jpeg,defective
gs://pv-visual-inspection-ai-auto/visual_inspection/defective/defect_8.jpeg,defective
gs://pv-visual-inspection-ai-auto/visual_inspection/defective/defect_9.jpeg,defective
gs://pv-visual-inspection-ai-auto/visual_inspection/defective/defect_10.jpeg,defective


### Create the Dataset

Next, create the `Dataset` resource using the `create` method for the `ImageDataset` class, which takes the following parameters:

- `display_name`: The human readable name for the `Dataset` resource.
- `gcs_source`: A list of one or more dataset index files to import the data items into the `Dataset` resource.
- `import_schema_uri`: The data labeling schema for the data items.

This operation may take several minutes.

In [18]:
dataset = aiplatform.ImageDataset.create(
    display_name="visualinspectionauto",
    gcs_source=[IMPORT_FILE],
    import_schema_uri=aiplatform.schema.dataset.ioformat.image.single_label_classification,
)

print(dataset.resource_name)

Creating ImageDataset
Create ImageDataset backing LRO: projects/999012933750/locations/us-central1/datasets/2038958551803625472/operations/7100144508716711936
ImageDataset created. Resource name: projects/999012933750/locations/us-central1/datasets/2038958551803625472
To use this ImageDataset in another session:
ds = aiplatform.ImageDataset('projects/999012933750/locations/us-central1/datasets/2038958551803625472')
Importing ImageDataset data: projects/999012933750/locations/us-central1/datasets/2038958551803625472
Import ImageDataset data backing LRO: projects/999012933750/locations/us-central1/datasets/2038958551803625472/operations/4570951105483112448
ImageDataset data imported. Resource name: projects/999012933750/locations/us-central1/datasets/2038958551803625472
projects/999012933750/locations/us-central1/datasets/2038958551803625472


### Create and run training pipeline

To train an AutoML model, you perform two steps: 1) create a training pipeline, and 2) run the pipeline.

#### Create training pipeline

An AutoML training pipeline is created with the `AutoMLImageTrainingJob` class, with the following parameters:

- `display_name`: The human readable name for the `TrainingJob` resource.
- `prediction_type`: The type task to train the model for.
  - `classification`: An image classification model.
  - `object_detection`: An image object detection model.
- `multi_label`: If a classification task, whether single (`False`) or multi-labeled (`True`).
- `model_type`: The type of model for deployment.
  - `CLOUD`: Deployment on Google Cloud
  - `CLOUD_HIGH_ACCURACY_1`: Optimized for accuracy over latency for deployment on Google Cloud.
  - `CLOUD_LOW_LATENCY_`: Optimized for latency over accuracy for deployment on Google Cloud.
  - `MOBILE_TF_VERSATILE_1`: Deployment on an edge device.
  - `MOBILE_TF_HIGH_ACCURACY_1`:Optimized for accuracy over latency for deployment on an edge device.
  - `MOBILE_TF_LOW_LATENCY_1`: Optimized for latency over accuracy for deployment on an edge device.
- `base_model`: (optional) Transfer learning from existing `Model` resource -- supported for image classification only.

The instantiated object is the DAG (directed acyclic graph) for the training job.

In [19]:
dag = aiplatform.AutoMLImageTrainingJob(
    display_name="visualinspectionauto",
    prediction_type="classification",
    multi_label=False,
    model_type="MOBILE_TF_LOW_LATENCY_1",
    base_model=None,
)

print(dag)

#### Run the training pipeline

Next, you run the DAG to start the training job by invoking the method `run`, with the following parameters:

- `dataset`: The `Dataset` resource to train the model.
- `model_display_name`: The human readable name for the trained model.
- `training_fraction_split`: The percentage of the dataset to use for training.
- `test_fraction_split`: The percentage of the dataset to use for test (holdout data).
- `validation_fraction_split`: The percentage of the dataset to use for validation.
- `budget_milli_node_hours`: (optional) Maximum training time specified in unit of millihours (1000 = hour).
- `disable_early_stopping`: If `True`, training maybe completed before using the entire budget if the service believes it cannot further improve on the model objective measurements.

The `run` method when completed returns the `Model` resource.

The execution of the training pipeline will take around 2 hours

In [20]:
model = dag.run(
    dataset=dataset,
    model_display_name="visualinspectionauto",
    training_fraction_split=0.8,
    validation_fraction_split=0.1,
    test_fraction_split=0.1,
    budget_milli_node_hours=1000,
    disable_early_stopping=False,
)

View Training:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/625206913126105088?project=999012933750
AutoMLImageTrainingJob projects/999012933750/locations/us-central1/trainingPipelines/625206913126105088 current state:
PipelineState.PIPELINE_STATE_RUNNING
AutoMLImageTrainingJob projects/999012933750/locations/us-central1/trainingPipelines/625206913126105088 current state:
PipelineState.PIPELINE_STATE_RUNNING
AutoMLImageTrainingJob projects/999012933750/locations/us-central1/trainingPipelines/625206913126105088 current state:
PipelineState.PIPELINE_STATE_RUNNING
AutoMLImageTrainingJob projects/999012933750/locations/us-central1/trainingPipelines/625206913126105088 current state:
PipelineState.PIPELINE_STATE_RUNNING
AutoMLImageTrainingJob projects/999012933750/locations/us-central1/trainingPipelines/625206913126105088 current state:
PipelineState.PIPELINE_STATE_RUNNING
AutoMLImageTrainingJob projects/999012933750/locations/us-central1/trainingPipelines/6252

## Review model evaluation scores

After your model training has finished, you can review the evaluation scores for it using the `list_model_evaluations()` method. This method will return an iterator for each evaluation slice.

In [21]:
model_evaluations = model.list_model_evaluations()

for model_evaluation in model_evaluations:
    print(model_evaluation.to_dict())

{'name': 'projects/999012933750/locations/us-central1/models/6723955607524605952@1/evaluations/8326995663346728960', 'metricsSchemaUri': 'gs://google-cloud-aiplatform/schema/modelevaluation/classification_metrics_1.0.0.yaml', 'metrics': {'logLoss': 0.098869264, 'auPrc': 1.0, 'confidenceMetrics': [{'precision': 0.5, 'recall': 1.0}, {'precision': 0.5, 'confidenceThreshold': 0.05, 'recall': 1.0}, {'precision': 0.6666667, 'confidenceThreshold': 0.1, 'recall': 1.0}, {'recall': 1.0, 'precision': 1.0, 'confidenceThreshold': 0.15}, {'confidenceThreshold': 0.2, 'recall': 1.0, 'precision': 1.0}, {'confidenceThreshold': 0.25, 'precision': 1.0, 'recall': 1.0}, {'precision': 1.0, 'confidenceThreshold': 0.3, 'recall': 1.0}, {'precision': 1.0, 'recall': 1.0, 'confidenceThreshold': 0.35}, {'precision': 1.0, 'confidenceThreshold': 0.4, 'recall': 1.0}, {'confidenceThreshold': 0.45, 'precision': 1.0, 'recall': 1.0}, {'recall': 1.0, 'precision': 1.0, 'confidenceThreshold': 0.5}, {'confidenceThreshold': 0.

## Export as Edge model

You can export an AutoML image object detection model as a `Edge` model which you can then custom deploy to an edge device or download locally. Use the method `export_model()` to export the model to Cloud Storage, which takes the following parameters:

- `artifact_destination`: The Cloud Storage location to store the SavedFormat model artifacts to.
- `export_format_id`: The format to save the model format as. For AutoML image object detection there is just one option:
   - `tf-saved-model`: TensorFlow SavedFormat for deployment to a container.
   - `tflite`: TensorFlow Lite for deployment to an edge or mobile device.
   - `edgetpu-tflite`: TensorFlow Lite for TPU
   - `tf-js`: TensorFlow for web client
   - `coral-ml`: for Coral devices

- `sync`: Whether to perform operational sychronously or asynchronously.

In [22]:
response = model.export_model(
    artifact_destination=BUCKET_URI, export_format_id="tflite", sync=True
)

model_package = response["artifactOutputUri"]

Exporting Model model: projects/999012933750/locations/us-central1/models/6723955607524605952
Export Model model backing LRO: projects/999012933750/locations/us-central1/models/6723955607524605952/operations/2822850762621583360
Model model exported. Resource name: projects/999012933750/locations/us-central1/models/6723955607524605952


#### Download the TFLite model artifacts

Now that you have an exported TFLite version of your model, you can test the exported model locally, but first downloading it from Cloud Storage.

In [23]:
! gsutil ls $model_package
# Download the model artifacts
! gsutil cp -r $model_package tflite

tflite_path = "tflite/model.tflite"

gs://automl-vision-workshop-pv-mlops-demo/model-6723955607524605952/tflite/2023-04-19T06:20:59.181313Z/model.tflite
Copying gs://automl-vision-workshop-pv-mlops-demo/model-6723955607524605952/tflite/2023-04-19T06:20:59.181313Z/model.tflite...
/ [1 files][549.9 KiB/549.9 KiB]                                                
Operation completed over 1 objects/549.9 KiB.                                    


#### Instantiate a TFLite interpreter

The TFLite version of the model is not a TensorFlow SavedModel format. You cannot directly use methods like predict(). Instead, one uses the TFLite interpreter. You must first setup the interpreter for the TFLite model as follows:

- Instantiate an TFLite interpreter for the TFLite model.
- Instruct the interpreter to allocate input and output tensors for the model.
- Get detail information about the models input and output tensors that will need to be known for prediction.

In [24]:
import tensorflow as tf

interpreter = tf.lite.Interpreter(model_path=tflite_path)
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
input_shape = input_details[0]["shape"]

print("input tensor shape", input_shape)

2023-04-19 06:21:26.775022: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-19 06:21:27.101958: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-04-19 06:21:31.061568: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/cuda/lib:/usr/local/lib/x86_64-linux-gnu:/usr/local/nvidia/lib:/usr/local/nvidia/lib64:/usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-04-19 06:21:31.061843: W tensorflow/strea

input tensor shape [  1 224 224   3]


### Get test item

You will use an arbitrary example out of the dataset as a test item. Don't be concerned that the example was likely used in training the model -- we just want to demonstrate how to make a prediction.

In [66]:
test_items = ! gsutil cat $IMPORT_FILE | head -n16
test_item = test_items[0].split(",")[0]  #defective
#test_item = test_items[15].split(",")[0] #ok
with tf.io.gfile.GFile(test_item, "rb") as f:
    content = f.read()
test_image = tf.io.decode_jpeg(content)
print("test image shape", test_image.shape)

test_image = tf.image.resize(test_image, (224, 224))
print("test image shape", test_image.shape, test_image.dtype)

test_image = tf.cast(test_image, dtype=tf.uint8).numpy()

test image shape (419, 448, 3)
test image shape (224, 224, 3) <dtype: 'float32'>


#### Make a prediction with TFLite model

Finally, you do a prediction using your TFLite model, as follows:

- Convert the test image into a batch of a single image (`np.expand_dims`)
- Set the input tensor for the interpreter to your batch of a single image (`data`).
- Invoke the interpreter.
- Retrieve the softmax probabilities for the prediction (`get_tensor`).
- Determine which label had the highest probability (`np.argmax`).

In [67]:
import numpy as np

data = np.expand_dims(test_image, axis=0)

interpreter.set_tensor(input_details[0]["index"], data)

interpreter.invoke()

softmax = interpreter.get_tensor(output_details[0]["index"])

label = np.argmax(softmax)

print(label)

0


# Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial.

In [ ]:
# delete_bucket = False

# # Delete the dataset using the Vertex dataset object
# dataset.delete()

# # Delete the model using the Vertex model object
# model.delete()

# # Delete the AutoML trainig job
# dag.delete()

# if delete_bucket or os.getenv("IS_TESTING"):
#     ! gsutil rm -r $BUCKET_URI